In [4]:
import sys
import cv2
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QMenuBar, QAction, 
                             QWidget, QVBoxLayout, QHBoxLayout, QLabel, QFrame, 
                             QGridLayout, QListWidget)
from PyQt5.QtGui import QImage, QPixmap, QPainter, QColor, QFont
from PyQt5.QtCore import QTimer, QTime
import time
import torch
import mysql.connector
import pandas as pd

def get_db_connection():
        try:
            connection = mysql.connector.connect(
                host="localhost",
                user="root",
                password="",  # Your MySQL password here
                database="viz_optilytics"
            )
            return connection
        except mysql.connector.Error as err:
            print(f"Error: {err}")
            return None

######################################################################################
# Load color reference CSV and create color lookup
index = ["color", "color_name", "hex", "R", "G", "B"]
csv = pd.read_csv('colors.csv', names=index, header=None)

def get_color_name(bgr):
    b, g, r = bgr
    minimum = 10000
    cname = "Unknown"
    for i in range(len(csv)):
        d = abs(r - int(csv.loc[i, "R"])) + abs(g - int(csv.loc[i, "G"])) + abs(b - int(csv.loc[i, "B"]))
        if d <= minimum:
            minimum = d
            cname = csv.loc[i, "color_name"]
    return cname

# Function to store vehicle information in the database
def store_vehicle_info(color, vehicle_type, model, timestamp):
    conn = get_db_connection()
    cursor = conn.cursor()

    sql = "INSERT INTO vehicles (color, type, model, timestamp) VALUES (%s, %s, %s, %s)"
    values = (color, vehicle_type, model, timestamp)

    cursor.execute(sql, values)
    conn.commit()
    cursor.close()
    conn.close()

# Function to extract color from bounding box
def extract_color(frame, bbox):
    x1, y1, x2, y2 = bbox
    roi = frame[y1:y2, x1:x2]
    average_color = np.mean(roi, axis=(0, 1))
    color_name = get_color_name(average_color)
    return color_name
###############################################        

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Camera Feed and Surveillance System")
        self.setGeometry(100, 100, 1200, 800)  # Increased window size

        # Create the menu bar
        self.menu_bar = self.menuBar()
        file_menu = self.menu_bar.addMenu("File")
        
        exit_action = QAction("Exit", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # Create a central widget and layout
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)

        # Add a top section for the alert indicator
        self.alert_section = QFrame()
        self.alert_section.setFixedHeight(50)
        self.alert_section.setStyleSheet("background-color: #f2f2f2; border-bottom: 2px solid #cccccc;")
        self.alert_label = QLabel(self.alert_section)
        self.alert_label.setGeometry(10, 10, 40, 40)  # Adjusted circle position and size
        self.alert_label.setStyleSheet("border-radius: 20px; background-color: green;")  # Green initially
        alert_title = QLabel("Alert Status", self.alert_section)
        alert_title.setFont(QFont("Arial", 12, QFont.Bold))
        alert_title.setGeometry(60, 10, 200, 40)
        main_layout.addWidget(self.alert_section)

        # Create a horizontal layout for left and right sections
        content_layout = QHBoxLayout()

        # Create a vertical section on the left
        left_section = QFrame()
        left_section.setStyleSheet("background-color: #e6e6e6; border-right: 2px solid #cccccc;")
        left_section.setFixedWidth(200)
        left_layout = QVBoxLayout(left_section)

        left_title = QLabel("Optimal videos")
        left_title.setFont(QFont("Arial", 14, QFont.Bold))
        left_layout.addWidget(left_title)
#################################################################################
        self.video_list = QListWidget()
        left_layout.addWidget(self.video_list)
        
          # Load videos from database
        self.load_videos()
#################################################################################3
        
        left_layout.addWidget(QLabel("Item 1"))
        left_layout.addWidget(QLabel("Item 2"))

        content_layout.addWidget(left_section)

        # Create a grid layout for the right section
        right_section = QGridLayout()

        # Section 1: Live Camera Feed
        self.camera_section = QFrame()
        self.camera_section.setStyleSheet("border: 1px solid #000000;")
        self.camera_label = QLabel("Camera Feed")
        self.camera_label.setFixedSize(500, 375)  # Larger feed size
        self.camera_layout = QVBoxLayout(self.camera_section)
        self.camera_layout.addWidget(self.camera_label)
        right_section.addWidget(self.camera_section, 0, 0)

        # Section 2: Object Detection
        self.detection_section = QFrame()
        self.detection_section.setStyleSheet("border: 1px solid #000000;")
        self.detection_label = QLabel("Detected Objects")
        self.detection_label.setFixedSize(500, 375)
        self.detection_layout = QVBoxLayout(self.detection_section)
        self.detection_layout.addWidget(self.detection_label)
        right_section.addWidget(self.detection_section, 0, 1)

        # Section 3: Graphs
        self.graph_section_1 = QFrame()
        self.graph_section_1.setStyleSheet("border: 1px solid #000000;")
        self.graph_layout_1 = QVBoxLayout(self.graph_section_1)
        self.graph_layout_1.addWidget(QLabel("Graph 1"))
        right_section.addWidget(self.graph_section_1, 1, 0)

        # Section 4: More Graphs
        self.graph_section_2 = QFrame()
        self.graph_section_2.setStyleSheet("border: 1px solid #000000;")
        self.graph_layout_2 = QVBoxLayout(self.graph_section_2)
        self.graph_layout_2.addWidget(QLabel("Graph 2"))
        right_section.addWidget(self.graph_section_2, 1, 1)

        content_layout.addLayout(right_section)
        main_layout.addLayout(content_layout)

        # Initialize YOLOv5, tracking data, and camera
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5 model

        self.tracked_objects = {}  # To store object ID and time of detection
        self.alert_active = False  # Keep track of alert state
        self.init_camera()
        

    def init_camera(self):
        """Initialize the camera feed."""
        self.cap = cv2.VideoCapture(0)  # Open the default camera
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(20)  # Update the frame every 20 ms

    def update_frame(self):
        """Update the camera feed frame."""
        ret, frame = self.cap.read()
        if ret:
            # Convert the frame to RGB format for display
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            h, w, ch = frame.shape
            bytes_per_line = ch * w
            qt_image = QImage(frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
            self.camera_label.setPixmap(QPixmap.fromImage(qt_image))

            # Process frame for object detection
            self.process_detections(frame)

            # Update the alert state
            self.update_alert()

            

    def load_videos(self):
        connection = get_db_connection()
        if not connection:
            QMessageBox.critical(self, "Database Error", "Could not connect to the database.")
            return

        cursor = connection.cursor()
        cursor.execute("SELECT id, video_name FROM video_storage")
        videos = cursor.fetchall()

        cursor.close()
        connection.close()

        for video in videos:
            self.video_list.addItem(f"{video[0]}: {video[1]}")
            
      # Function to check if the vehicle is static
    def is_static(self, current_position, last_position, threshold=10):
        # Check if the bounding box coordinates have not changed significantly
        return (abs(current_position[0] - last_position[0]) < threshold and
                abs(current_position[1] - last_position[1]) < threshold and
                abs(current_position[2] - last_position[2]) < threshold and
                abs(current_position[3] - last_position[3]) < threshold)
            
    def process_detections(self, frame):
        """Detect objects in the frame using YOLOv5 and track presence of persons and vehicles."""
        results = self.model(frame)  # Perform YOLOv5 detection
        detections = results.xyxy[0].numpy()  # Get bounding boxes and labels

        current_time = time.time()
        person_detected = False
        vehicle_detected = False

        for detection in detections:
            x1, y1, x2, y2, conf, class_id = detection
            class_id = int(class_id)
            label = self.model.names[class_id]

            # Logic for person detection
            if label == 'person' and conf > 0.3:  # Adjust confidence threshold as needed
                person_detected = True
                object_id = "person"  # Simple ID for tracking person
                color = (0, 255, 0)  # Green for first detection
                # Track detection time
                if object_id not in self.tracked_objects:
                    self.tracked_objects[object_id] = current_time  # Store the time of first detection
                else:
                    elapsed_time = current_time - self.tracked_objects[object_id]

                    # Generate alert if person has been detected for more than 5 seconds
                    if elapsed_time > 5:
                        color = (0, 0, 255)  # Red for alert
                        self.alert_active = True
                        # Add alert handling logic here
                    else:
                        color = (0, 255, 0)  # Green for normal detection
                        self.alert_active = False

                # Draw bounding box and label
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                cv2.putText(frame, f"{label} {int(conf * 100)}%", (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            elif label in ['car', 'truck', 'bus'] and conf > 0.3:  # Adjust confidence threshold as needed
                vehicle_detected = True
                object_id = label  # Use vehicle type as ID for tracking

                # Track detection time
                current_position = (int(x1), int(y1), int(x2), int(y2))  # Store current bounding box coordinates

                if object_id not in self.tracked_objects:
                    self.tracked_objects[object_id] = {
                        'first_detection_time': current_time,
                        'last_position': current_position,
                        'elapsed_time': 0
                    }
                    color = (0, 255, 0)  # Green for first detection
                else:
                    tracked_data = self.tracked_objects[object_id]
                    elapsed_time = current_time - tracked_data['first_detection_time']

                    # Check if the position is the same (within a threshold)
                    if self.is_static(current_position, tracked_data['last_position']):
                        tracked_data['elapsed_time'] = elapsed_time

                        # Generate alert if vehicle has been static for more than 5 seconds
                        if tracked_data['elapsed_time'] > 5:
                            color = (0, 0, 255)  # Red for alert
                            self.alert_active = True
                            # Add alert handling logic here
                        else:
                            color = (0, 255, 0)  # Green for normal detection
                            self.alert_active = False
                    else:
                        # Reset the detection time if the vehicle has moved
                        tracked_data['first_detection_time'] = current_time
                        tracked_data['last_position'] = current_position
                        tracked_data['elapsed_time'] = 0  # Reset elapsed time

                # Draw bounding box and label
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                cv2.putText(frame, f"{label} {int(conf * 100)}%", (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)


        # If no person or vehicle is detected, reset tracking
        if not person_detected:
            self.tracked_objects.pop("person", None)  # Remove person ID if not detected
        if not vehicle_detected:
            for vehicle_type in ['car', 'truck', 'bus']:
                self.tracked_objects.pop(vehicle_type, None)  # Remove vehicle IDs if not detected

        # Update the QPixmap in the detection label
        qt_image = QImage(frame.data, frame.shape[1], frame.shape[0], QImage.Format_RGB888)
        self.detection_label.setPixmap(QPixmap.fromImage(qt_image))

    
    def update_alert(self):
        """Update the alert indicator based on the presence of people/vehicles for too long."""
        if self.alert_active:
            self.alert_label.setStyleSheet("border-radius: 20px; background-color: red;")  # Red when alert is active
        else:
            self.alert_label.setStyleSheet("border-radius: 20px; background-color: green;")  # Green when no alert

    def closeEvent(self, event):
        """Handle the event when the window is closed."""
        self.timer.stop()  # Stop the timer
        self.cap.release()  # Release the camera
        cv2.destroyAllWindows()  # Close any OpenCV windows
        event.accept()  # Accept the event and proceed with the closing

def main():
    app = QApplication(sys.argv)
    
    window = MainWindow()
    window.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


UnboundLocalError: cannot access local variable 'color' where it is not associated with a value

UnboundLocalError: cannot access local variable 'color' where it is not associated with a value

UnboundLocalError: cannot access local variable 'color' where it is not associated with a value

UnboundLocalError: cannot access local variable 'color' where it is not associated with a value

...

UnboundLocalError: cannot access local variable 'color' where it is not associated with a value

UnboundLocalError: cannot access local variable 'color' where it is not associated with a value

UnboundLocalError: cannot access local variable 'color' where it is not associated with a value

UnboundLocalError: cannot access local variable 'color' where it is not associated with a value

SystemExit: 0

### tHIS CODE IS WORKING FOR ONLY ONE PERSON DETECTION

In [ ]:
# import cv2
# import torch
# import time

# # Load YOLOv5 model (using pre-trained model)
# model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True, device='cpu')
# model.classes = [0]  # Only detect people (class 0)

# # Initialize the list of trackers using the legacy module
# trackers = cv2.legacy.MultiTracker_create()

# # Dictionary to store the time of entry for each person
# person_times = {}
# stay_threshold = 10  # seconds
# grace_period = 2  # seconds

# # Start capturing video
# cap = cv2.VideoCapture(0)

# def create_tracker(frame, bbox):
#     tracker = cv2.legacy.TrackerCSRT_create()  # Use CSRT tracker
#     tracker.init(frame, tuple(bbox))
#     return tracker

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     frame = cv2.resize(frame, (640, 480))  # Resize frame to reduce memory load

#     # Run YOLOv5 inference
#     results = model(frame)
    
#     # Extract bounding boxes and confidences
#     detections = []
#     for det in results.xyxy[0]:  # For each detection
#         x1, y1, x2, y2 = map(int, det[:4])
#         bbox = [x1, y1, x2 - x1, y2 - y1]  # Convert to (x, y, w, h)
#         detections.append(bbox)
    
#     # If no trackers are active, create new trackers for each detection
#     if len(trackers.getObjects()) == 0:
#         for bbox in detections:
#             trackers.add(create_tracker(frame, bbox), frame, tuple(bbox))

#     # Update the trackers for each person
#     success, boxes = trackers.update(frame)

#     current_time = time.time()

#     # Loop over the tracked bounding boxes
#     for i, box in enumerate(boxes):
#         x, y, w, h = map(int, box)
#         person_id = i + 1  # Use a simple counter for person ID

#         # Draw bounding boxes
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
#         cv2.putText(frame, f'Person {person_id}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#         # Track the time a person has stayed in the frame
#         if person_id not in person_times:
#             person_times[person_id] = {'start_time': current_time, 'grace_timer': None}

#         person_data = person_times[person_id]
#         time_in_frame = current_time - person_data['start_time']

#         if time_in_frame > stay_threshold:
#             print(f"Alert! Person {person_id} stayed in frame for {time_in_frame:.2f} seconds.")
    
#     # Display the output frame
#     cv2.imshow('Person Detection and Tracking', frame)

#     # Exit on pressing 'q'
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


## tHIS CODE IS WORKING FOR people detection corectly but is not integrating IN THE MAJOR CODE

In [3]:
# import torch
# import cv2
# import time

# # Load YOLOv5 model (pretrained on COCO)
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# # Set model to only detect the "person" class
# model.classes = [0]  # 0 is the class index for "person" in COCO

# # Load video or camera stream
# cap = cv2.VideoCapture(0)  # Change 0 to video path if needed

# # Variables for tracking people in the frame
# person_enter_time = {}
# person_in_frame = set()  # Set to keep track of persons currently in the frame
# grace_period = 2  # Time a person needs to be detected to generate an alert (2 seconds)
# alert_threshold = 120  # Time in seconds to trigger the alert for prolonged presence

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Perform inference
#     results = model(frame)
    
#     # Extract the bounding boxes and check for person class (ID 0)
#     detections = results.xyxy[0].cpu().numpy()  # Raw detections as numpy array
#     persons_detected = []

#     # Count detected people (heads)
#     for det in detections:
#         class_id = int(det[5])  # Class ID is the 6th element in detection
#         if class_id == 0:  # 0 corresponds to 'person'
#             persons_detected.append(det[:4])  # Append bounding box coordinates for persons
    
#     current_time = time.time()
#     head_count = len(persons_detected)  # Count the number of detected people (heads)

#     # Handle new detections
#     for i, bbox in enumerate(persons_detected):
#         if i not in person_in_frame:
#             person_enter_time[i] = current_time  # Track when the person enters the frame
#             person_in_frame.add(i)  # Mark person as in frame
#             print(f"Person {i} detected at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_time))}")

#         # Check if person has been in the frame for longer than alert threshold
#         duration = current_time - person_enter_time[i]
#         if duration > grace_period:
#             print(f"Alert: Person {i} has been in the frame for {duration:.2f} seconds. Head count: {head_count}")
#         if duration > alert_threshold:
#             print(f"Warning: Person {i} has exceeded {alert_threshold} seconds! Head count: {head_count}")

#     # Remove persons no longer detected (after the grace period)
#     exited = set()
#     for person_id in person_in_frame:
#         if person_id >= len(persons_detected) or current_time - person_enter_time[person_id] > grace_period:
#             exited.add(person_id)
#             print(f"Person {person_id} left the frame at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_time))}")

#     person_in_frame.difference_update(exited)

#     # Render detections on the frame
#     frame = results.render()[0]

#     # Show frame with detections
#     cv2.imshow('Person Detection', frame)

#     # Press 'q' to exit
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release resources
# cap.release()
# cv2.destroyAllWindows()


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Person 0 detected at 2024-10-11 23:40:44
Person 0 left the frame at 2024-10-11 23:40:45
Person 0 detected at 2024-10-11 23:41:03
Person 0 left the frame at 2024-10-11 23:41:04
Person 0 detected at 2024-10-11 23:41:09
Person 0 left the frame at 2024-10-11 23:41:10
Person 0 detected at 2024-10-11 23:41:13
Person 0 left the frame at 2024-10-11 23:41:15
Person 0 detected at 2024-10-11 23:41:16
Alert: Person 0 has been in the frame for 2.35 seconds. Head count: 1
Person 0 left the frame at 2024-10-11 23:41:19
Person 0 detected at 2024-10-11 23:41:19
Alert: Person 0 has been in the frame for 2.22 seconds. Head count: 1
Person 0 left the frame at 2024-10-11 23:41:22
Person 0 detected at 2024-10-11 23:41:22
Alert: Person 0 has been in the frame for 2.15 seconds. Head count: 1
Person 0 left the frame at 2024-10-11 23:41:24
Person 0 detected at 2024-10-11 23:41:25
Person 0 left the frame at 2024-10-11 23:41:26
Person 0 detected at 2024-10-11 23:41:27
Alert: Person 0 has been in the frame for 2.5

## PEOPLE DETECTION ADD ALL THE DATA TO THE DATABASE

In [ ]:
# import torch
# import cv2
# import time
# import mysql.connector
# import requests
# import webbrowser

# # Load YOLOv5 model (pretrained on COCO)
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# # Set model to only detect the "person" class
# model.classes = [0]

# # Load video or camera stream
# cap = cv2.VideoCapture(0)

# # Database connection
# conn = mysql.connector.connect(
#     host='localhost',
#     user='root',
#     password='',
#     database='viz_optilytics'
# )

# cursor = conn.cursor()

# def get_camera_location():
#     api_url = 'https://api.opencagedata.com/geocode/v1/json'
#     api_key = 'YOUR_API_KEY'  # Replace with your API key
#     query = 'Your City, Country'  # Replace with a valid query
#     response = requests.get(f'{api_url}?q={query}&key={api_key}')
#     if response.status_code == 200:
#         data = response.json()
#         if data['results']:
#             lat = data['results'][0]['geometry']['lat']
#             lng = data['results'][0]['geometry']['lng']
#             print(f"Retrieved coordinates: Latitude = {lat}, Longitude = {lng}")
#             return lat, lng
#     print("Failed to retrieve coordinates.")
#     return None, None

# # Variables for tracking people in the frame
# person_enter_time = {}
# person_in_frame = set()  
# alert_threshold = 120  
# alert_triggered = set()  
# grace_period = 5  # Time (seconds) after which a person will be considered as "gone"
# buffer_time = 5   # Time (seconds) to keep track of a person who has left the frame

# # Time of last detection for each person
# buffer_exit_time = {}  

# # Get camera location
# latitude, longitude = get_camera_location()

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Perform inference
#     results = model(frame)

#     # Extract the bounding boxes and check for person class (ID 0)
#     detections = results.xyxy[0].cpu().numpy()
#     persons_detected = []

#     # Count detected people (heads)
#     for det in detections:
#         class_id = int(det[5])  
#         if class_id == 0:  
#             persons_detected.append(det[:4])  

#     current_time = time.time()
#     head_count = len(persons_detected)

#     # Handle new detections
#     for i in range(head_count):
#         if i not in person_in_frame:
#             person_enter_time[i] = current_time
#             person_in_frame.add(i)
#             buffer_exit_time[i] = current_time  # Initialize exit time
#             print(f"Person {i} detected at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_time))}")

#         # Check if person has been in the frame for longer than grace period
#         duration = current_time - person_enter_time[i]
#         if duration > grace_period and i not in alert_triggered:
#             print(f"Alert: Person {i} has been in the frame for {duration:.2f} seconds. Head count: {head_count}")
#             alert_triggered.add(i)  

#             # Insert data into the database
#             cursor.execute('''INSERT INTO people (id, people_count, timestamp, location_lat, location_long)
#                               VALUES (%s, %s, %s, %s, %s)''',
#                            (i, head_count, time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_time)), latitude, longitude))
#             conn.commit()

#             if latitude is not None and longitude is not None:
#                 map_url = f"https://www.google.com/maps?q={latitude},{longitude}"
#                 webbrowser.open(map_url)
#             else:
#                 print("Cannot open map: Invalid location coordinates.")

#     # Remove persons no longer detected
#     exited = set()
#     for person_id in person_in_frame:
#         if person_id >= head_count:
#             # Check if the person has been out of the frame for longer than the grace period
#             if current_time - buffer_exit_time[person_id] > grace_period:
#                 exited.add(person_id)  
#                 alert_triggered.discard(person_id)  
#                 print(f"Person {person_id} left the frame at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_time))}")
#         else:
#             buffer_exit_time[person_id] = current_time  # Update last seen time

#     person_in_frame.difference_update(exited)

#     # Render detections on the frame
#     frame = results.render()[0]

#     # Show frame with detections
#     cv2.imshow('Person Detection', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release resources
# cap.release()
# cv2.destroyAllWindows()
# cursor.close()
# conn.close()


## VEHICLE DETECTION

In [1]:
from PyQt5 import QtWidgets, QtGui, QtCore
import sys
import cv2
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage, QPixmap
import torch
from datetime import datetime
import time
import numpy as np


class VizOptilyticsApp(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Viz Optilytics")
        self.setGeometry(0, 0, 1200, 700)

        # Initialize QTimer (fixing the timer error)
        self.timer = QTimer(self)  # Initialize the QTimer object

        # Initialize video capture object
        self.video_capture = None

        # Detection states
        self.detecting_person = False
        self.detecting_vehicle = False

         # Track detected persons and duration
        self.person_detected_time = None  # To track when a person is first detected
        self.alert_generated = False  # To track if alert is already generated
        
        # Load YOLOv5 model
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5 model

         # Initialize alert labels for dynamic updates
        self.alert_label = None
        self.duration_label = None
        
        self.grace_period_duration = 5  # 5 seconds grace period

        
        self.vehicle_positions = {}  # Stores vehicle ID -> (last position, detection start time)
        self.stationary_alert_generated = {}  # To track if stationary alert has been generated

        
        self.initUI()
    def initUI(self):
        # Main Layout
        main_layout = QtWidgets.QVBoxLayout(self)
        main_layout.setContentsMargins(50, 50, 50, 30)

        # Top Section: Title and Menu
        top_bar = QtWidgets.QHBoxLayout()
        top_bar.addStretch(1)

        title = QtWidgets.QLabel("Viz Optilytics")
        title.setFont(QtGui.QFont("Arial", 28, QtGui.QFont.Bold))
        title.setStyleSheet("color: #FFD700;")
        title.setAlignment(QtCore.Qt.AlignCenter)

        top_bar.addWidget(title)
        top_bar.addStretch(1)

        menu_button = QtWidgets.QPushButton("☰")
        menu_button.setStyleSheet("background-color: #333333; color: #FFD700; font-size: 20px; border-radius: 10px;")
        menu_button.setFixedSize(50, 50)
        menu_button.setMenu(self.create_menu())

        top_bar.addWidget(menu_button)
        main_layout.addLayout(top_bar)

        title_spacer = QtWidgets.QSpacerItem(20, 40, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Fixed)
        main_layout.addItem(title_spacer)

        # Horizontal layout split: Videos, Buttons, Alerts, etc.
        content_layout = QtWidgets.QHBoxLayout()
        content_layout.setSpacing(20)

        # Video Buttons and Info Section
        main_vertical_layout = QtWidgets.QVBoxLayout()

        video_list_layout = QtWidgets.QVBoxLayout()
        recorded_videos_label = QtWidgets.QLabel("Recorded Videos")
        recorded_videos_label.setFont(QtGui.QFont("Arial", 16, QtGui.QFont.Bold))
        recorded_videos_label.setStyleSheet("color: white;")
        video_list_layout.addWidget(recorded_videos_label)

        # Video buttons
        for i in range(3):
            video_btn = QtWidgets.QPushButton(f"Video {i + 1}")
            video_btn.setStyleSheet("background-color: #555555; color: white; border-radius: 8px; padding: 8px;")
            video_btn.setFixedWidth(200)
            video_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
            video_btn.clicked.connect(lambda checked, i=i: self.play_video(i + 1))  # Connect button to function
            video_list_layout.addWidget(video_btn)

        main_vertical_layout.addLayout(video_list_layout)

        detection_info_widget = QtWidgets.QWidget()
        detection_info_layout = QtWidgets.QVBoxLayout(detection_info_widget)
        detection_info_layout.setSpacing(0)
        detection_info_widget.setStyleSheet("background-color: #444444; border-radius: 10px;")

        date_time_label = QtWidgets.QLabel("Date: 2024-10-11\nTime: 12:45 PM")
        date_time_label.setStyleSheet("color: white; font-size: 14px; padding: 8px;")
        detection_info_layout.addWidget(date_time_label)

        detection_status = QtWidgets.QLabel("Status: Vehicle Detected")
        detection_status.setStyleSheet("color: white; font-size: 14px; padding: 8px;")
        detection_info_layout.addWidget(detection_status)
        main_vertical_layout.addWidget(detection_info_widget)

        # Real-time streaming button
        real_time_btn = QtWidgets.QPushButton("Real-Time Streaming")
        real_time_btn.setStyleSheet("background-color: #666666; padding: 10px; font-size: 14px; border-radius: 8px;")
        real_time_btn.setFixedWidth(200)
        real_time_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        real_time_btn.clicked.connect(self.start_real_time_stream)
        main_vertical_layout.addWidget(real_time_btn)

        # Analytics button
        analytics_btn = QtWidgets.QPushButton("Analytics")
        analytics_btn.setStyleSheet("background-color: #666666; padding: 10px; font-size: 14px; border-radius: 8px;")
        analytics_btn.setFixedWidth(200)
        analytics_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        analytics_btn.clicked.connect(self.show_analytics)
        main_vertical_layout.addWidget(analytics_btn)

        content_layout.addLayout(main_vertical_layout)

        # Video display area and other buttons
        main_left_layout = QtWidgets.QVBoxLayout()

        video_layout = QtWidgets.QVBoxLayout()
        video_controls_layout = QtWidgets.QHBoxLayout()

        # Create a layout to position the close button at the top-right corner
        close_button = QtWidgets.QPushButton("X")
        close_button.setStyleSheet("background-color: red; color: white; font-size: 18px; border-radius: 10px;")
        close_button.setFixedSize(30, 30)
        close_button.clicked.connect(self.stop_stream)

        # Create a wrapper for the video feed and the button
        video_feed_wrapper = QtWidgets.QWidget()
        video_feed_layout = QtWidgets.QVBoxLayout(video_feed_wrapper)
        video_feed_layout.setContentsMargins(0, 0, 0, 0)  # Remove margins for precise alignment

        # Create a layout for the close button at the top-right corner
        close_button_layout = QtWidgets.QHBoxLayout()
        close_button_layout.addStretch(1)  # Add stretch to push the button to the right
        close_button_layout.addWidget(close_button)

        video_feed_layout.addLayout(close_button_layout)  # Add the close button layout
        self.video_feed = QtWidgets.QLabel("Video Stream")
        self.video_feed.setAlignment(QtCore.Qt.AlignCenter)
        self.video_feed.setStyleSheet("background-color: white; color: black; border: 2px solid;")
        self.video_feed.setFixedSize(970, 480)  # Set a fixed size for the video feed
        self.video_feed.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Expanding)

        video_feed_layout.addWidget(self.video_feed)  # Add the video feed below the button

        # Add the video feed wrapper to the main video layout
        video_layout.addWidget(video_feed_wrapper)

        # Detection buttons
        detection_buttons_layout = QtWidgets.QHBoxLayout()
        record_btn = QtWidgets.QPushButton("Record")
        record_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        record_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        record_btn.clicked.connect(self.record_options)
        detection_buttons_layout.addWidget(record_btn)

       # Assigning person_detection_btn as an instance variable
        self.person_detection_btn = QtWidgets.QPushButton("Person Detection")
        self.person_detection_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        self.person_detection_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        self.person_detection_btn.clicked.connect(self.toggle_person_detection)
        detection_buttons_layout.addWidget(self.person_detection_btn)

        self.vehicle_detection_btn = QtWidgets.QPushButton("Vehicle Detection")
        self.vehicle_detection_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        self.vehicle_detection_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        self.vehicle_detection_btn.clicked.connect(self.toggle_vehicle_detection)
        detection_buttons_layout.addWidget(self.vehicle_detection_btn)

        video_layout.addLayout(detection_buttons_layout)
        main_left_layout.addLayout(video_layout)

        # Alerts section
        alerts_layout = QtWidgets.QHBoxLayout()
        self.alert_label = QtWidgets.QLabel("Alert: No Detection")
        self.alert_label.setFixedSize(450, 37)  # Set fixed size for the alert label
        self.alert_label.setAlignment(QtCore.Qt.AlignCenter)
        self.alert_label.setStyleSheet("background-color: green; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        alerts_layout.addWidget(self.alert_label)

        self.duration_label = QtWidgets.QLabel("Time Duration: 00:00:00")
        self.duration_label.setAlignment(QtCore.Qt.AlignCenter)
        self.duration_label.setFixedSize(450, 37)  # Set fixed size for the alert label
        self.duration_label.setStyleSheet("background-color: green; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        alerts_layout.addWidget(self.duration_label)

        main_left_layout.addLayout(alerts_layout)
        content_layout.addLayout(main_left_layout)

        main_layout.addLayout(content_layout)
        self.setStyleSheet("background-color: #222222; color: white;")

        self.setLayout(main_layout)

    def create_menu(self):
        menu = QtWidgets.QMenu(self)
        menu.addAction("Option 1", self.option1_action)
        menu.addAction("Option 2", self.option2_action)
        return menu

    def play_video(self, video_id):
        print(f"Playing Video {video_id}")

    def start_real_time_stream(self):
        print("Starting real-time video stream")
        # Start video capture (assuming you have a webcam at index 0)
        self.video_capture = cv2.VideoCapture(0)
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(20)

    def stop_stream(self):
        print("Stopping video stream")
        if self.video_capture is not None:
            self.video_capture.release()
            self.video_capture = None
            self.video_feed.clear()  # Clear the video feed when stopping
            self.timer.stop()  # Stop the timer

            
    def update_frame(self):
        ret, frame = self.video_capture.read()
        if ret:
            # Convert the frame from BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Check if person detection is active
            if self.detecting_person:
                frame = self.detect_person(frame)  # Call person detection if active
            if self.detecting_vehicle:
                frame = self.detect_vehicle(frame)
                
            # Get the frame dimensions and convert to QImage
            h, w, ch = frame.shape
            bytes_per_line = ch * w
            qimg = QImage(frame.data, frame.shape[1], frame.shape[0], bytes_per_line, QImage.Format_RGB888)

            # Display the QImage on the QLabel (video_feed)
            self.video_feed.setPixmap(QPixmap.fromImage(qimg))


    def toggle_person_detection(self):
        self.detecting_person = not self.detecting_person  # Toggle the state
        if self.detecting_person:
            self.person_detection_btn.setText("Stop Person Detection")
        else:
            self.person_detection_btn.setText("Start Person Detection")
            self.person_detected_time = None  # Reset timer when detection stops
            self.alert_generated = False
            self.alert_label.setText("Alert: No Detection")
            self.alert_label.setStyleSheet("background-color: green; color: white;")

    def detect_person(self, frame):
        # Perform inference
        results = self.model(frame)

        # Get predictions
        predictions = results.pred[0]

        person_detected = False
        for *box, conf, cls in predictions:
            if int(cls) == 0:  # Class index for person
                person_detected = True
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame, f'Person {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if person_detected:
            self.update_person_detection_time()
        else:
            self.reset_person_detection()

        return frame

    def update_person_detection_time(self):
        """Track how long a person has been in the frame and update alert if needed."""
        if self.person_detected_time is None:
            self.person_detected_time = time.time()  # Start the timer

        duration = time.time() - self.person_detected_time

        # Format the duration
        hours, rem = divmod(duration, 3600)
        minutes, seconds = divmod(rem, 60)
        duration_str = "{:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds))

        # Update duration label
        self.duration_label.setText(f"Time Duration: {duration_str}")

        # Check if detection duration exceeds the threshold (e.g., 5 minutes or 300 seconds)
        if duration > 10 and not self.alert_generated:  # 300 seconds = 5 minutes
            self.alert_label.setText("ALERT: Person detected for over 5 minutes!")
            self.alert_label.setStyleSheet("background-color: red; color: white;")
            self.duration_label.setStyleSheet("background-color: red; color: white;")
            self.alert_generated = True

    def reset_person_detection(self):
        """Reset the detection if no person is found."""
        if self.person_detected_time is not None:
            self.person_detected_time = None  # Reset the timer
            self.duration_label.setText("Time Duration: 00:00:00")
            self.alert_label.setText("Alert: No Detection")
            self.alert_label.setStyleSheet("background-color: green; color: white;")
            self.duration_label.setStyleSheet("background-color: green; color: white;")
            self.alert_generated = False

            
#     def detect_vehicle(self, frame):
#     # Perform inference
#         results = self.model(frame)  # Use the loaded YOLOv5 model

#         # Get predictions
#         predictions = results.pred[0]  # Get the first result (frame-wise)

#         # Draw boxes for detected persons
#         for *box, conf, cls in predictions:
#             if int(cls) in [2, 3, 5, 7]: # Class index for   2: Car  3: Motorcycle  5: Bus   7: Truck
#                 x1, y1, x2, y2 = map(int, box)
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
#                 vehicle_type = ["Car", "Motorcycle", "Bus", "Truck"][int(cls) - 2]  # Assign the appropriate vehicle label
#                 cv2.putText(frame, f'{vehicle_type} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
#         return frame  # Return the modified frame with detections


    def detect_vehicle(self, frame):
        # Perform inference using YOLOv5 model
        results = self.model(frame)

        # Get predictions
        predictions = results.pred[0]

        current_time = time.time()  # Get the current time

        for *box, conf, cls in predictions:
            if int(cls) in [2, 3, 5, 7]:  # Vehicle class: Car (2), Motorcycle (3), Bus (5), Truck (7)
                x1, y1, x2, y2 = map(int, box)
                vehicle_type = ["Car", "Motorcycle", "Bus", "Truck"][int(cls) - 2]

                # Calculate the center of the bounding box
                vehicle_center = ((x1 + x2) // 2, (y1 + y2) // 2)

                # Use the center position to track vehicle movement
                if vehicle_type not in self.vehicle_positions:
                    # If this is a new vehicle, start tracking it
                    self.vehicle_positions[vehicle_type] = (vehicle_center, current_time)
                    self.stationary_alert_generated[vehicle_type] = False
                else:
                    # Get the previous position and detection start time
                    prev_position, start_time = self.vehicle_positions[vehicle_type]

                    # Calculate movement distance between current and previous position
                    movement_distance = np.linalg.norm(np.array(vehicle_center) - np.array(prev_position))

                    # Define a movement threshold (e.g., 20 pixels)
                    movement_threshold = 20

                    if movement_distance < movement_threshold:
                        # If the vehicle has not moved significantly, check how long it's been stationary
                        stationary_duration = current_time - start_time

                        # Update the time duration label
                        self.duration_label.setText(f"Vehicle {vehicle_type} stationary for {int(stationary_duration)} seconds")

                        # Check if the stationary duration exceeds 5 minutes (300 seconds)
                        if stationary_duration > 10 and not self.stationary_alert_generated[vehicle_type]:
                            self.alert_label.setText(f"ALERT: {vehicle_type} stationary for over 5 minutes!")
                            self.alert_label.setStyleSheet("background-color: red; color: white;")
                            self.duration_label.setStyleSheet("background-color: red; color: white;")
                            self.stationary_alert_generated[vehicle_type] = True  # Mark that the alert has been generated
                    else:
                        # If the vehicle moved, reset the detection start time
                        self.vehicle_positions[vehicle_type] = (vehicle_center, current_time)
                        self.stationary_alert_generated[vehicle_type] = False

                # Draw the bounding box and label the vehicle
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{vehicle_type} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        return frame


    
    def toggle_vehicle_detection(self):
        """Toggle vehicle detection on and off."""
        self.detecting_vehicle = not self.detecting_vehicle  # Toggle the state
        if self.detecting_vehicle:
            self.vehicle_detection_btn.setText("Stop vehicle Detection")
        else:
            self.vehicle_detection_btn.setText("Start vehicle Detection")

            
    

    def show_analytics(self):
        print("Showing analytics")

    def record_options(self):
        print("Recording options selected")

    def option1_action(self):
        print("Option 1 selected")

    def option2_action(self):
        print("Option 2 selected")


if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    window = VizOptilyticsApp()
    window.show()
    sys.exit(app.exec_())


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


SystemExit: 0

In [1]:
from PyQt5 import QtWidgets, QtGui, QtCore
import sys
import cv2
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage, QPixmap
import torch
from datetime import datetime
import time
import numpy as np


class VizOptilyticsApp(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Viz Optilytics")
        self.setGeometry(0, 0, 1200, 700)

        # Initialize QTimer (fixing the timer error)
        self.timer = QTimer(self)  # Initialize the QTimer object

        # Initialize video capture object
        self.video_capture = None

        # Detection states
        self.detecting_person = False
        self.detecting_vehicle = False

         # Track detected persons and duration
        self.person_detected_time = None  # To track when a person is first detected
        self.alert_generated = False  # To track if alert is already generated
        
        # Load YOLOv5 model
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5 model

         # Initialize alert labels for dynamic updates
        self.alert_label = None
        self.duration_label = None
        
        self.detection_grace_period = 10  # Number of frames to maintain detection box
        self.frames_since_last_detection = 0
        self.last_detection_box = None  # Store the last detected bounding box

        
        self.vehicle_positions = {}  # Stores vehicle ID -> (last position, detection start time)
        self.stationary_alert_generated = {}  # To track if stationary alert has been generated

        self.video1 = False
        self.video2 = False
        self.video3 = False
        self.real_time_stream = False

        self.initUI()
    def initUI(self):
        # Main Layout
        main_layout = QtWidgets.QVBoxLayout(self)
        main_layout.setContentsMargins(50, 50, 50, 30)

        # Top Section: Title and Menu
        top_bar = QtWidgets.QHBoxLayout()
        top_bar.addStretch(1)

        title = QtWidgets.QLabel("Viz Optilytics")
        title.setFont(QtGui.QFont("Arial", 28, QtGui.QFont.Bold))
        title.setStyleSheet("color: #FFD700;")
        title.setAlignment(QtCore.Qt.AlignCenter)

        top_bar.addWidget(title)
        top_bar.addStretch(1)

        menu_button = QtWidgets.QPushButton("☰")
        menu_button.setStyleSheet("background-color: #333333; color: #FFD700; font-size: 20px; border-radius: 10px;")
        menu_button.setFixedSize(50, 50)
        menu_button.setMenu(self.create_menu())

        top_bar.addWidget(menu_button)
        main_layout.addLayout(top_bar)

        title_spacer = QtWidgets.QSpacerItem(20, 40, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Fixed)
        main_layout.addItem(title_spacer)

        # Horizontal layout split: Videos, Buttons, Alerts, etc.
        content_layout = QtWidgets.QHBoxLayout()
        content_layout.setSpacing(20)

        # Video Buttons and Info Section
        main_vertical_layout = QtWidgets.QVBoxLayout()

        video_list_layout = QtWidgets.QVBoxLayout()
        recorded_videos_label = QtWidgets.QLabel("Recorded Videos")
        recorded_videos_label.setFont(QtGui.QFont("Arial", 16, QtGui.QFont.Bold))
        recorded_videos_label.setStyleSheet("color: white;")
        video_list_layout.addWidget(recorded_videos_label)

        # Video buttons
        for i in range(3):
            video_btn = QtWidgets.QPushButton(f"Video {i + 1}")
            video_btn.setStyleSheet("background-color: #555555; color: white; border-radius: 8px; padding: 8px;")
            video_btn.setFixedWidth(200)
            video_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)

            # Connect each button to a separate function based on the index
            if i == 0:
                video_btn.clicked.connect(self.play_video_1)
            elif i == 1:
                video_btn.clicked.connect(self.play_video_2)
            elif i == 2:
                video_btn.clicked.connect(self.play_video_3)

            video_list_layout.addWidget(video_btn)

        main_vertical_layout.addLayout(video_list_layout)

        detection_info_widget = QtWidgets.QWidget()
        detection_info_layout = QtWidgets.QVBoxLayout(detection_info_widget)
        detection_info_layout.setSpacing(0)
        detection_info_widget.setStyleSheet("background-color: #444444; border-radius: 10px;")

        date_time_label = QtWidgets.QLabel("Date: 2024-10-11\nTime: 12:45 PM")
        date_time_label.setStyleSheet("color: white; font-size: 14px; padding: 8px;")
        detection_info_layout.addWidget(date_time_label)

        detection_status = QtWidgets.QLabel("Status: Vehicle Detected")
        detection_status.setStyleSheet("color: white; font-size: 14px; padding: 8px;")
        detection_info_layout.addWidget(detection_status)
        main_vertical_layout.addWidget(detection_info_widget)

        # Real-time streaming button
        real_time_btn = QtWidgets.QPushButton("Real-Time Streaming")
        real_time_btn.setStyleSheet("background-color: #666666; padding: 10px; font-size: 14px; border-radius: 8px;")
        real_time_btn.setFixedWidth(200)
        real_time_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        real_time_btn.clicked.connect(self.start_real_time_stream)
        main_vertical_layout.addWidget(real_time_btn)

        # Analytics button
        analytics_btn = QtWidgets.QPushButton("Analytics")
        analytics_btn.setStyleSheet("background-color: #666666; padding: 10px; font-size: 14px; border-radius: 8px;")
        analytics_btn.setFixedWidth(200)
        analytics_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        analytics_btn.clicked.connect(self.show_analytics)
        main_vertical_layout.addWidget(analytics_btn)

        content_layout.addLayout(main_vertical_layout)

        # Video display area and other buttons
        main_left_layout = QtWidgets.QVBoxLayout()

        video_layout = QtWidgets.QVBoxLayout()
        video_controls_layout = QtWidgets.QHBoxLayout()

        # Create a layout to position the close button at the top-right corner
        close_button = QtWidgets.QPushButton("X")
        close_button.setStyleSheet("background-color: red; color: white; font-size: 18px; border-radius: 10px;")
        close_button.setFixedSize(30, 30)
        close_button.clicked.connect(self.stop_stream)

        # Create a wrapper for the video feed and the button
        video_feed_wrapper = QtWidgets.QWidget()
        video_feed_layout = QtWidgets.QVBoxLayout(video_feed_wrapper)
        video_feed_layout.setContentsMargins(0, 0, 0, 0)  # Remove margins for precise alignment

        # Create a layout for the close button at the top-right corner
        close_button_layout = QtWidgets.QHBoxLayout()
        close_button_layout.addStretch(1)  # Add stretch to push the button to the right
        close_button_layout.addWidget(close_button)

        video_feed_layout.addLayout(close_button_layout)  # Add the close button layout
        self.video_feed = QtWidgets.QLabel("Video Stream")
        self.video_feed.setAlignment(QtCore.Qt.AlignCenter)
        self.video_feed.setStyleSheet("background-color: white; color: black; border: 2px solid;")
        self.video_feed.setFixedSize(970, 480)  # Set a fixed size for the video feed
        self.video_feed.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Expanding)

        video_feed_layout.addWidget(self.video_feed)  # Add the video feed below the button

        # Add the video feed wrapper to the main video layout
        video_layout.addWidget(video_feed_wrapper)

        # Detection buttons
        detection_buttons_layout = QtWidgets.QHBoxLayout()
        record_btn = QtWidgets.QPushButton("Record")
        record_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        record_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        record_btn.clicked.connect(self.record_options)
        detection_buttons_layout.addWidget(record_btn)

       # Assigning person_detection_btn as an instance variable
        self.person_detection_btn = QtWidgets.QPushButton("Person Detection")
        self.person_detection_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        self.person_detection_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        self.person_detection_btn.clicked.connect(self.toggle_person_detection)
        detection_buttons_layout.addWidget(self.person_detection_btn)

        self.vehicle_detection_btn = QtWidgets.QPushButton("Vehicle Detection")
        self.vehicle_detection_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        self.vehicle_detection_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        self.vehicle_detection_btn.clicked.connect(self.toggle_vehicle_detection)
        detection_buttons_layout.addWidget(self.vehicle_detection_btn)

        video_layout.addLayout(detection_buttons_layout)
        main_left_layout.addLayout(video_layout)

        # Alerts section
        alerts_layout = QtWidgets.QHBoxLayout()
        self.alert_label = QtWidgets.QLabel("Alert: No Detection")
        self.alert_label.setFixedSize(450, 37)  # Set fixed size for the alert label
        self.alert_label.setAlignment(QtCore.Qt.AlignCenter)
        self.alert_label.setStyleSheet("background-color: green; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        alerts_layout.addWidget(self.alert_label)

        self.duration_label = QtWidgets.QLabel("Time Duration: 00:00:00")
        self.duration_label.setAlignment(QtCore.Qt.AlignCenter)
        self.duration_label.setFixedSize(450, 37)  # Set fixed size for the alert label
        self.duration_label.setStyleSheet("background-color: green; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        alerts_layout.addWidget(self.duration_label)

        main_left_layout.addLayout(alerts_layout)
        content_layout.addLayout(main_left_layout)

        main_layout.addLayout(content_layout)
        self.setStyleSheet("background-color: #222222; color: white;")

        self.setLayout(main_layout)

    def create_menu(self):
        menu = QtWidgets.QMenu(self)
        menu.addAction("Option 1", self.option1_action)
        menu.addAction("Option 2", self.option2_action)
        return menu

    def start_real_time_stream(self):
        print("Starting real-time video stream")
        self.real_time_stream=True
        # Start video capture (assuming you have a webcam at index 0)
        self.video_display()  # Call video_feed to initiate video capture
        
    def video_display(self):
        if self.real_time_stream:
            self.video_capture = cv2.VideoCapture(0)
        elif self.video1:
            self.video_capture = cv2.VideoCapture(r"C:\Users\mahnoor\Downloads\853889-hd_1920_1080_25fps.mp4")
        elif self.video2:
            self.video_capture = cv2.VideoCapture(r"C:\Users\mahnoor\Downloads\2273134-hd_1280_720_30fps.mp4")
        elif self.video3:
            self.video_capture = cv2.VideoCapture(r"C:\Users\mahnoor\Downloads\12494476_3840_2160_50fps.mp4")
            
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(20)

    def stop_stream(self):
        print("Stopping video stream")
        if self.video_capture is not None:
            self.video_capture.release()
            self.video_capture = None
            self.video_feed.clear()  # Clear the video feed when stopping
            self.timer.stop()  # Stop the timer

            
    def update_frame(self):
        ret, frame = self.video_capture.read()
        if ret:
            # Convert the frame from BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Check if person detection is active
            if self.detecting_person:
                frame = self.detect_person(frame)  # Call person detection if active
            if self.detecting_vehicle:
                frame = self.detect_vehicle(frame)
                
            # Get the frame dimensions and convert to QImage
            h, w, ch = frame.shape
            bytes_per_line = ch * w
            qimg = QImage(frame.data, frame.shape[1], frame.shape[0], bytes_per_line, QImage.Format_RGB888)

            # Display the QImage on the QLabel (video_feed)
            self.video_feed.setPixmap(QPixmap.fromImage(qimg))


    def toggle_person_detection(self):
        self.detecting_person = not self.detecting_person  # Toggle the state
        if self.detecting_person:
            self.person_detection_btn.setText("Stop Person Detection")
        else:
            self.person_detection_btn.setText("Start Person Detection")
            self.person_detected_time = None  # Reset timer when detection stops
            self.alert_generated = False
            self.alert_label.setText("Alert: No Detection")
            self.alert_label.setStyleSheet("background-color: green; color: white;")

    def detect_person(self, frame):
        # Perform inference
        results = self.model(frame)

        # Get predictions
        predictions = results.pred[0]

        person_detected = False
        for *box, conf, cls in predictions:
            if int(cls) == 0:  # Class index for person
                person_detected = True
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame, f'Person {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if person_detected:
            self.update_person_detection_time()
        else:
            self.reset_person_detection()

        return frame

    def update_person_detection_time(self):
        """Track how long a person has been in the frame and update alert if needed."""
        if self.person_detected_time is None:
            self.person_detected_time = time.time()  # Start the timer

        duration = time.time() - self.person_detected_time

        # Format the duration
        hours, rem = divmod(duration, 3600)
        minutes, seconds = divmod(rem, 60)
        duration_str = "{:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds))

        # Update duration label
        self.duration_label.setText(f"Time Duration: {duration_str}")

        # Check if detection duration exceeds the threshold (e.g., 5 minutes or 300 seconds)
        if duration > 10 and not self.alert_generated:  # 300 seconds = 5 minutes
            self.alert_label.setText("ALERT: Person detected for over 5 minutes!")
            self.alert_label.setStyleSheet("background-color: red; color: white;")
            self.duration_label.setStyleSheet("background-color: red; color: white;")
            self.alert_generated = True

    def reset_person_detection(self):
        """Reset the detection if no person is found."""
        if self.person_detected_time is not None:
            self.person_detected_time = None  # Reset the timer
            self.duration_label.setText("Time Duration: 00:00:00")
            self.alert_label.setText("Alert: No Detection")
            self.alert_label.setStyleSheet("background-color: green; color: white;")
            self.duration_label.setStyleSheet("background-color: green; color: white;")
            self.alert_generated = False

            
    def detect_vehicle(self, frame):
        # Perform inference using YOLOv5 model
        results = self.model(frame)

        # Get predictions
        predictions = results.pred[0]

        current_time = time.time()  # Get the current time

        vehicle_detected = False  # Flag to check if any vehicle is detected

        for *box, conf, cls in predictions:
            if int(cls) in [2, 3, 5, 7]:  # Vehicle class: Car (2), Motorcycle (3), Bus (5), Truck (7)
                x1, y1, x2, y2 = map(int, box)
                vehicle_type = ["Car", "Motorcycle", "Bus", "Truck"][int(cls) - 2]

                # Calculate the center of the bounding box
                vehicle_center = ((x1 + x2) // 2, (y1 + y2) // 2)

                vehicle_detected = True  # Set the flag to True since we detected a vehicle

                # Use the center position to track vehicle movement
                if vehicle_type not in self.vehicle_positions:
                    # If this is a new vehicle, start tracking it
                    self.vehicle_positions[vehicle_type] = (vehicle_center, current_time)
                    self.stationary_alert_generated[vehicle_type] = False
                else:
                    # Get the previous position and detection start time
                    prev_position, start_time = self.vehicle_positions[vehicle_type]

                    # Calculate movement distance between current and previous position
                    movement_distance = np.linalg.norm(np.array(vehicle_center) - np.array(prev_position))

                    # Define a movement threshold (e.g., 20 pixels)
                    movement_threshold = 20

                    if movement_distance < movement_threshold:
                        # If the vehicle has not moved significantly, check how long it's been stationary
                        stationary_duration = current_time - start_time

                        # Update the time duration label
                        self.duration_label.setText(f"Vehicle {vehicle_type} stationary for {int(stationary_duration)} seconds")

                        # Check if the stationary duration exceeds 5 minutes (300 seconds)
                        if stationary_duration > 5 and not self.stationary_alert_generated[vehicle_type]:
                            self.alert_label.setText(f"ALERT: {vehicle_type} stationary for over 5 minutes!")
                            self.alert_label.setStyleSheet("background-color: red; color: white;")
                            self.duration_label.setStyleSheet("background-color: red; color: white;")
                            self.stationary_alert_generated[vehicle_type] = True  # Mark that the alert has been generated
                    else:
                        # If the vehicle moved, reset the detection start time
                        self.vehicle_positions[vehicle_type] = (vehicle_center, current_time)
                        self.stationary_alert_generated[vehicle_type] = False

                # Draw the bounding box and label the vehicle
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{vehicle_type} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # If no vehicle is detected, reset the alert and duration labels
        if not vehicle_detected:
            self.alert_label.setText("No vehicle detected")
            self.alert_label.setStyleSheet("background-color: green; color: white;")
            self.duration_label.setText("No detection duration: 00 seconds")
            self.duration_label.setStyleSheet("background-color: green; color: white;")

        return frame


    def toggle_vehicle_detection(self):
        """Toggle vehicle detection on and off."""
        self.detecting_vehicle = not self.detecting_vehicle  # Toggle the state
        if self.detecting_vehicle:
            self.vehicle_detection_btn.setText("Stop vehicle Detection")
        else:
            self.vehicle_detection_btn.setText("Start vehicle Detection")

            
    

   # Define the three separate play video functions
    def play_video_1(self):
        print("Playing Video 1")
        self.real_time_stream=False
        self.video1 = True  # Set instance variable
        self.video2 = False  # Reset other video flags
        self.video3 = False
        self.stop_stream()  # Stop any current stream before starting a video
        self.video_display()  # Call video_feed to start playing the video

    def play_video_2(self):
        print("Playing Video 2")
        self.real_time_stream=False
        self.video1 = False  # Reset other video flags
        self.video2 = True
        self.video3 = False
        self.stop_stream()  # Stop any current stream before starting a video
        self.video_display()  # Call video_feed to start playing the video

    def play_video_3(self):
        print("Playing Video 3")
        self.real_time_stream=False
        self.video1 = False  # Reset other video flags
        self.video2 = False
        self.video3 = True
        self.stop_stream()  # Stop any current stream before starting a video
        self.video_display()  # Call video_feed to start playing the video

    # Add your logic to play video 3 here
    def show_analytics(self):
        print("Showing analytics")

    def record_options(self):
        print("Recording options selected")

    def option1_action(self):
        print("Option 1 selected")

    def option2_action(self):
        print("Option 2 selected")


if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    window = VizOptilyticsApp()
    window.show()
    sys.exit(app.exec_())


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Playing Video 1
Stopping video stream
Stopping video stream


SystemExit: 0

In [ ]:
from PyQt5 import QtWidgets, QtGui, QtCore
import sys
import cv2
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage, QPixmap
import torch
from datetime import datetime
import time
import numpy as np


class VizOptilyticsApp(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Viz Optilytics")
        self.setGeometry(0, 0, 1200, 700)

        # Initialize QTimer (fixing the timer error)
        self.timer = QTimer(self)  # Initialize the QTimer object

        # Initialize video capture object
        self.video_capture = None

        # Detection states
        self.detecting_person = False
        self.detecting_vehicle = False

         # Track detected persons and duration
        self.person_detected_time = None  # To track when a person is first detected
        self.alert_generated = False  # To track if alert is already generated
        
        # Load YOLOv5 model
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5 model

         # Initialize alert labels for dynamic updates
        self.alert_label = None
        self.duration_label = None
        
        self.detection_grace_period = 10  # Number of frames to maintain detection box
        self.frames_since_last_detection = 0
        self.last_detection_box = None  # Store the last detected bounding box

        
        self.vehicle_positions = {}  # Stores vehicle ID -> (last position, detection start time)
        self.stationary_alert_generated = {}  # To track if stationary alert has been generated

        self.video1 = False
        self.video2 = False
        self.video3 = False
        self.real_time_stream = False

        self.initUI()
    def initUI(self):
        # Main Layout
        main_layout = QtWidgets.QVBoxLayout(self)
        main_layout.setContentsMargins(50, 50, 50, 30)

        # Top Section: Title and Menu
        top_bar = QtWidgets.QHBoxLayout()
        top_bar.addStretch(1)

        title = QtWidgets.QLabel("Viz Optilytics")
        title.setFont(QtGui.QFont("Arial", 28, QtGui.QFont.Bold))
        title.setStyleSheet("color: #FFD700;")
        title.setAlignment(QtCore.Qt.AlignCenter)

        top_bar.addWidget(title)
        top_bar.addStretch(1)

        menu_button = QtWidgets.QPushButton("☰")
        menu_button.setStyleSheet("background-color: #333333; color: #FFD700; font-size: 20px; border-radius: 10px;")
        menu_button.setFixedSize(50, 50)
        menu_button.setMenu(self.create_menu())

        top_bar.addWidget(menu_button)
        main_layout.addLayout(top_bar)

        title_spacer = QtWidgets.QSpacerItem(20, 40, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Fixed)
        main_layout.addItem(title_spacer)

        # Horizontal layout split: Videos, Buttons, Alerts, etc.
        content_layout = QtWidgets.QHBoxLayout()
        content_layout.setSpacing(20)

        # Video Buttons and Info Section
        main_vertical_layout = QtWidgets.QVBoxLayout()

        video_list_layout = QtWidgets.QVBoxLayout()
        recorded_videos_label = QtWidgets.QLabel("Recorded Videos")
        recorded_videos_label.setFont(QtGui.QFont("Arial", 16, QtGui.QFont.Bold))
        recorded_videos_label.setStyleSheet("color: white;")
        video_list_layout.addWidget(recorded_videos_label)

        # Video buttons
        for i in range(3):
            video_btn = QtWidgets.QPushButton(f"Video {i + 1}")
            video_btn.setStyleSheet("background-color: #555555; color: white; border-radius: 8px; padding: 8px;")
            video_btn.setFixedWidth(200)
            video_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)

            # Connect each button to a separate function based on the index
            if i == 0:
                video_btn.clicked.connect(self.play_video_1)
            elif i == 1:
                video_btn.clicked.connect(self.play_video_2)
            elif i == 2:
                video_btn.clicked.connect(self.play_video_3)

            video_list_layout.addWidget(video_btn)

        main_vertical_layout.addLayout(video_list_layout)

        detection_info_widget = QtWidgets.QWidget()
        detection_info_layout = QtWidgets.QVBoxLayout(detection_info_widget)
        detection_info_layout.setSpacing(0)
        detection_info_widget.setStyleSheet("background-color: #444444; border-radius: 10px;")

        date_time_label = QtWidgets.QLabel("Date: 2024-10-11\nTime: 12:45 PM")
        date_time_label.setStyleSheet("color: white; font-size: 14px; padding: 8px;")
        detection_info_layout.addWidget(date_time_label)

        detection_status = QtWidgets.QLabel("Status: Vehicle Detected")
        detection_status.setStyleSheet("color: white; font-size: 14px; padding: 8px;")
        detection_info_layout.addWidget(detection_status)
        main_vertical_layout.addWidget(detection_info_widget)

        # Real-time streaming button
        real_time_btn = QtWidgets.QPushButton("Real-Time Streaming")
        real_time_btn.setStyleSheet("background-color: #666666; padding: 10px; font-size: 14px; border-radius: 8px;")
        real_time_btn.setFixedWidth(200)
        real_time_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        real_time_btn.clicked.connect(self.start_real_time_stream)
        main_vertical_layout.addWidget(real_time_btn)

        # Analytics button
        analytics_btn = QtWidgets.QPushButton("Analytics")
        analytics_btn.setStyleSheet("background-color: #666666; padding: 10px; font-size: 14px; border-radius: 8px;")
        analytics_btn.setFixedWidth(200)
        analytics_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        analytics_btn.clicked.connect(self.show_analytics)
        main_vertical_layout.addWidget(analytics_btn)

        content_layout.addLayout(main_vertical_layout)

        # Video display area and other buttons
        main_left_layout = QtWidgets.QVBoxLayout()

        video_layout = QtWidgets.QVBoxLayout()
        video_controls_layout = QtWidgets.QHBoxLayout()

        # Create a layout to position the close button at the top-right corner
        close_button = QtWidgets.QPushButton("X")
        close_button.setStyleSheet("background-color: red; color: white; font-size: 18px; border-radius: 10px;")
        close_button.setFixedSize(30, 30)
        close_button.clicked.connect(self.stop_stream)

        # Create a wrapper for the video feed and the button
        video_feed_wrapper = QtWidgets.QWidget()
        video_feed_layout = QtWidgets.QVBoxLayout(video_feed_wrapper)
        video_feed_layout.setContentsMargins(0, 0, 0, 0)  # Remove margins for precise alignment

        # Create a layout for the close button at the top-right corner
        close_button_layout = QtWidgets.QHBoxLayout()
        close_button_layout.addStretch(1)  # Add stretch to push the button to the right
        close_button_layout.addWidget(close_button)

        video_feed_layout.addLayout(close_button_layout)  # Add the close button layout
        self.video_feed = QtWidgets.QLabel("Video Stream")
        self.video_feed.setAlignment(QtCore.Qt.AlignCenter)
        self.video_feed.setStyleSheet("background-color: white; color: black; border: 2px solid;")
        self.video_feed.setFixedSize(970, 480)  # Set a fixed size for the video feed
        self.video_feed.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Expanding)

        video_feed_layout.addWidget(self.video_feed)  # Add the video feed below the button

        # Add the video feed wrapper to the main video layout
        video_layout.addWidget(video_feed_wrapper)

        # Detection buttons
        detection_buttons_layout = QtWidgets.QHBoxLayout()
        record_btn = QtWidgets.QPushButton("Record")
        record_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        record_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        record_btn.clicked.connect(self.record_options)
        detection_buttons_layout.addWidget(record_btn)

       # Assigning person_detection_btn as an instance variable
        self.person_detection_btn = QtWidgets.QPushButton("Person Detection")
        self.person_detection_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        self.person_detection_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        self.person_detection_btn.clicked.connect(self.toggle_person_detection)
        detection_buttons_layout.addWidget(self.person_detection_btn)

        self.vehicle_detection_btn = QtWidgets.QPushButton("Vehicle Detection")
        self.vehicle_detection_btn.setStyleSheet("background-color: #666666; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        self.vehicle_detection_btn.setSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        self.vehicle_detection_btn.clicked.connect(self.toggle_vehicle_detection)
        detection_buttons_layout.addWidget(self.vehicle_detection_btn)

        video_layout.addLayout(detection_buttons_layout)
        main_left_layout.addLayout(video_layout)

        # Alerts section
        alerts_layout = QtWidgets.QHBoxLayout()
        self.alert_label = QtWidgets.QLabel("Alert: No Detection")
        self.alert_label.setFixedSize(450, 37)  # Set fixed size for the alert label
        self.alert_label.setAlignment(QtCore.Qt.AlignCenter)
        self.alert_label.setStyleSheet("background-color: green; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        alerts_layout.addWidget(self.alert_label)

        self.duration_label = QtWidgets.QLabel("Time Duration: 00:00:00")
        self.duration_label.setAlignment(QtCore.Qt.AlignCenter)
        self.duration_label.setFixedSize(450, 37)  # Set fixed size for the alert label
        self.duration_label.setStyleSheet("background-color: green; color: white; padding: 10px; font-size: 14px; border-radius: 8px;")
        alerts_layout.addWidget(self.duration_label)

        main_left_layout.addLayout(alerts_layout)
        content_layout.addLayout(main_left_layout)

        main_layout.addLayout(content_layout)
        self.setStyleSheet("background-color: #222222; color: white;")

        self.setLayout(main_layout)

    def create_menu(self):
        menu = QtWidgets.QMenu(self)
        menu.addAction("Option 1", self.option1_action)
        menu.addAction("Option 2", self.option2_action)
        return menu

    def start_real_time_stream(self):
        print("Starting real-time video stream")
        self.real_time_stream=True
        # Start video capture (assuming you have a webcam at index 0)
        self.video_display()  # Call video_feed to initiate video capture
        
    def video_display(self):
        if self.real_time_stream:
            self.video_capture = cv2.VideoCapture(0)
        elif self.video1:
            self.video_capture = cv2.VideoCapture(r"C:\Users\mahnoor\Downloads\853889-hd_1920_1080_25fps.mp4")
        elif self.video2:
            self.video_capture = cv2.VideoCapture(r"C:\Users\mahnoor\Downloads\2273134-hd_1280_720_30fps.mp4")
        elif self.video3:
            self.video_capture = cv2.VideoCapture(r"C:\Users\mahnoor\Downloads\12494476_3840_2160_50fps.mp4")
            
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(20)

    def stop_stream(self):
        print("Stopping video stream")
        if self.video_capture is not None:
            self.video_capture.release()
            self.video_capture = None
            self.video_feed.clear()  # Clear the video feed when stopping
            self.timer.stop()  # Stop the timer

            
    def update_frame(self):
        ret, frame = self.video_capture.read()
        if ret:
            # Convert the frame from BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Check if person detection is active
            if self.detecting_person:
                frame = self.detect_person(frame)  # Call person detection if active
            if self.detecting_vehicle:
                frame = self.detect_vehicle(frame)
                
            # Get the frame dimensions and convert to QImage
            h, w, ch = frame.shape
            bytes_per_line = ch * w
            qimg = QImage(frame.data, frame.shape[1], frame.shape[0], bytes_per_line, QImage.Format_RGB888)

            # Display the QImage on the QLabel (video_feed)
            self.video_feed.setPixmap(QPixmap.fromImage(qimg))


    def toggle_person_detection(self):
        self.detecting_person = not self.detecting_person  # Toggle the state
        if self.detecting_person:
            self.person_detection_btn.setText("Stop Person Detection")
        else:
            self.person_detection_btn.setText("Start Person Detection")
            self.person_detected_time = None  # Reset timer when detection stops
            self.alert_generated = False
            self.alert_label.setText("Alert: No Detection")
            self.alert_label.setStyleSheet("background-color: green; color: white;")

    def detect_person(self, frame):
        # Perform inference
        results = self.model(frame)

        # Get predictions
        predictions = results.pred[0]

        person_detected = False
        for *box, conf, cls in predictions:
            if int(cls) == 0:  # Class index for person
                person_detected = True
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame, f'Person {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if person_detected:
            self.update_person_detection_time()
        else:
            self.reset_person_detection()

        return frame

    def update_person_detection_time(self):
        """Track how long a person has been in the frame and update alert if needed."""
        if self.person_detected_time is None:
            self.person_detected_time = time.time()  # Start the timer

        duration = time.time() - self.person_detected_time

        # Format the duration
        hours, rem = divmod(duration, 3600)
        minutes, seconds = divmod(rem, 60)
        duration_str = "{:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds))

        # Update duration label
        self.duration_label.setText(f"Time Duration: {duration_str}")

        # Check if detection duration exceeds the threshold (e.g., 5 minutes or 300 seconds)
        if duration > 10 and not self.alert_generated:  # 300 seconds = 5 minutes
            self.alert_label.setText("ALERT: Person detected for over 5 minutes!")
            self.alert_label.setStyleSheet("background-color: red; color: white;")
            self.duration_label.setStyleSheet("background-color: red; color: white;")
            self.alert_generated = True

    def reset_person_detection(self):
        """Reset the detection if no person is found."""
        if self.person_detected_time is not None:
            self.person_detected_time = None  # Reset the timer
            self.duration_label.setText("Time Duration: 00:00:00")
            self.alert_label.setText("Alert: No Detection")
            self.alert_label.setStyleSheet("background-color: green; color: white;")
            self.duration_label.setStyleSheet("background-color: green; color: white;")
            self.alert_generated = False

            
    def detect_vehicle(self, frame):
        # Perform inference using YOLOv5 model
        results = self.model(frame)

        # Get predictions
        predictions = results.pred[0]

        current_time = time.time()  # Get the current time

        vehicle_detected = False  # Flag to check if any vehicle is detected

        for *box, conf, cls in predictions:
            if int(cls) in [2, 3, 5, 7]:  # Vehicle class: Car (2), Motorcycle (3), Bus (5), Truck (7)
                x1, y1, x2, y2 = map(int, box)
                vehicle_type = ["Car", "Motorcycle", "Bus", "Truck"][int(cls) - 2]

                # Calculate the center of the bounding box
                vehicle_center = ((x1 + x2) // 2, (y1 + y2) // 2)

                vehicle_detected = True  # Set the flag to True since we detected a vehicle

                # Use the center position to track vehicle movement
                if vehicle_type not in self.vehicle_positions:
                    # If this is a new vehicle, start tracking it
                    self.vehicle_positions[vehicle_type] = (vehicle_center, current_time)
                    self.stationary_alert_generated[vehicle_type] = False
                else:
                    # Get the previous position and detection start time
                    prev_position, start_time = self.vehicle_positions[vehicle_type]

                    # Calculate movement distance between current and previous position
                    movement_distance = np.linalg.norm(np.array(vehicle_center) - np.array(prev_position))

                    # Define a movement threshold (e.g., 20 pixels)
                    movement_threshold = 20

                    if movement_distance < movement_threshold:
                        # If the vehicle has not moved significantly, check how long it's been stationary
                        stationary_duration = current_time - start_time

                        # Update the time duration label
                        self.duration_label.setText(f"Vehicle {vehicle_type} stationary for {int(stationary_duration)} seconds")

                        # Check if the stationary duration exceeds 5 minutes (300 seconds)
                        if stationary_duration > 5 and not self.stationary_alert_generated[vehicle_type]:
                            self.alert_label.setText(f"ALERT: {vehicle_type} stationary for over 5 minutes!")
                            self.alert_label.setStyleSheet("background-color: red; color: white;")
                            self.duration_label.setStyleSheet("background-color: red; color: white;")
                            self.stationary_alert_generated[vehicle_type] = True  # Mark that the alert has been generated
                    else:
                        # If the vehicle moved, reset the detection start time
                        self.vehicle_positions[vehicle_type] = (vehicle_center, current_time)
                        self.stationary_alert_generated[vehicle_type] = False

                # Draw the bounding box and label the vehicle
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{vehicle_type} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # If no vehicle is detected, reset the alert and duration labels
        if not vehicle_detected:
            self.alert_label.setText("No vehicle detected")
            self.alert_label.setStyleSheet("background-color: green; color: white;")
            self.duration_label.setText("No detection duration: 00 seconds")
            self.duration_label.setStyleSheet("background-color: green; color: white;")

        return frame


    def toggle_vehicle_detection(self):
        """Toggle vehicle detection on and off."""
        self.detecting_vehicle = not self.detecting_vehicle  # Toggle the state
        if self.detecting_vehicle:
            self.vehicle_detection_btn.setText("Stop vehicle Detection")
        else:
            self.vehicle_detection_btn.setText("Start vehicle Detection")

            
    

   # Define the three separate play video functions
    def play_video_1(self):
        print("Playing Video 1")
        self.real_time_stream=False
        self.video1 = True  # Set instance variable
        self.video2 = False  # Reset other video flags
        self.video3 = False
        self.stop_stream()  # Stop any current stream before starting a video
        self.video_display()  # Call video_feed to start playing the video

    def play_video_2(self):
        print("Playing Video 2")
        self.real_time_stream=False
        self.video1 = False  # Reset other video flags
        self.video2 = True
        self.video3 = False
        self.stop_stream()  # Stop any current stream before starting a video
        self.video_display()  # Call video_feed to start playing the video

    def play_video_3(self):
        print("Playing Video 3")
        self.real_time_stream=False
        self.video1 = False  # Reset other video flags
        self.video2 = False
        self.video3 = True
        self.stop_stream()  # Stop any current stream before starting a video
        self.video_display()  # Call video_feed to start playing the video
        
    # Add your logic to play video 3 here
    def show_analytics(self):
        print("Showing analytics")

    def record_options(self):
        print("Recording options selected")

    def option1_action(self):
        print("Option 1 selected")

    def option2_action(self):
        print("Option 2 selected")


if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    window = VizOptilyticsApp()
    window.show()
    sys.exit(app.exec_())
